In [1]:
%run /Users/denis/Documents/Projects/scripts/init.ipy

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
# Get LINCS dataframe
raw_dataframe = pd.DataFrame(json.loads(unicode(urllib2.urlopen('http://lincsportal.ccs.miami.edu/dcic/api/fetchdata?searchTerm=*&limit=2000').read(), errors='ignore'))['results']['documents'])
raw_dataframe.head()

,_version_,antibody,assaydesignmethod,assayformat,assayname,assayoverview,biologicalbucket,biologicalprocess,cellline,centerdatasetid,...,screeninglabinvestigator,size,smallmolecule,smlincsidentifier,statsfields,statsvalues,technologies,timepoints,tool,toollink
0,1571407499608719360,NaN,[KINOMEscan],Biochemical,[KINOMEscan kinase-small molecule binding assay],The KINOMEscan assay platform is based on a co...,Binding,[Small molecule metabolic process],NaN,http://lincs.hms.harvard.edu/db/datasets/20023/,...,Qingsong Liu,[ 0.07],[SB590885],"[LSM-42746, LSM-42746]","[smallmolecule, protein]","[1, 432]",KINOMEscan,[],"[Harmonizome, Life]",[http://amp.pharm.mssm.edu/Harmonizome/dataset...
1,1571407499677925376,NaN,[KINOMEscan],Biochemical,[KINOMEscan kinase-small molecule binding assay],The KINOMEscan assay platform is based on a co...,Binding,[Small molecule metabolic process],NaN,http://lincs.hms.harvard.edu/db/datasets/20032/,...,Qingsong Liu,[ 0.07],[GW843682X],"[LSM-1014, LSM-1014]","[smallmolecule, protein]","[1, 432]",KINOMEscan,[],"[Harmonizome, Life]",[http://amp.pharm.mssm.edu/Harmonizome/dataset...
2,1571407499740839936,NaN,[KINOMEscan],Biochemical,[KINOMEscan kinase-small molecule binding assay],The KINOMEscan assay platform is based on a co...,Binding,[Small molecule metabolic process],NaN,http://lincs.hms.harvard.edu/db/datasets/20031/,...,Qingsong Liu,[ 0.07],[GSK-461364],"[LSM-1013, LSM-1013]","[smallmolecule, protein]","[1, 432]",KINOMEscan,[],"[Harmonizome, Life]",[http://amp.pharm.mssm.edu/Harmonizome/dataset...
3,1571407499804803072,NaN,[KINOMEscan],Biochemical,[KINOMEscan kinase-small molecule binding assay],The KINOMEscan assay platform is based on a co...,Binding,[Small molecule metabolic process],NaN,http://lincs.hms.harvard.edu/db/datasets/20020/,...,Qingsong Liu,[ 0.07],[Sorafenib],"[LSM-1008, LSM-1008]","[smallmolecule, protein]","[1, 432]",KINOMEscan,[],"[Harmonizome, Life]",[http://amp.pharm.mssm.edu/Harmonizome/dataset...
4,1571407499867717632,NaN,[KINOMEscan],Biochemical,[KINOMEscan kinase-small molecule binding assay],The KINOMEscan assay platform is based on a co...,Binding,[Small molecule metabolic process],NaN,http://lincs.hms.harvard.edu/db/datasets/20021/,...,Qingsong Liu,[ 0.07],[HG6-64-1],"[LSM-43248, LSM-43248]","[smallmolecule, protein]","[1, 432]",KINOMEscan,[],"[Harmonizome, Life]",[http://amp.pharm.mssm.edu/Harmonizome/dataset...


In [3]:
raw_dataframe.to_csv('../data/lincs_datasets.txt', sep='\t')

In [4]:
# Get subset
lincs_dataframe = raw_dataframe[['ldplink', 'smallmolecule', 'technologies', 'cellline']].dropna().set_index('ldplink')

# Expand to list
dataframe_list = [lincs_dataframe[x].apply(pd.Series).stack().rename(x).reset_index().drop('level_1', axis=1).set_index('ldplink') for x in lincs_dataframe.columns]

# Merge
merged_dataframe = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), dataframe_list).drop_duplicates()

In [5]:
# Save
merged_dataframe.to_csv('../data/lincs_dataframe.txt', sep='\t')

In [6]:
merged_dataframe.shape

(522602, 3)